In [ ]:
import numpy as np 
import pandas as pd
from datetime import datetime
from typing import List, Callable
import matplotlib.pyplot as plt 
import seaborn as sns 

# Librerías para el procesamiento de access logs 
import re 
from parse import parse 
from lars.apache import ApacheSource, COMBINED, ApacheWarning

# Manejo de advertencias del sistema, usada para capturar las líneas que no pueden parsearse por problemas de lars (ApacheWarning)
import warnings

# Configuración de estilo para las gráficas
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

log_prep_path = '../data/target/access_log_master.csv'
df = pd.read_csv(log_prep_path)

In [ ]:
display(df['anomaly'].value_counts())
labeled_data = df[df['anomaly'] != -1]
unlabeled_data = df[df['anomaly'] == -1]

### Etiquetado Manual Simple 
Etiquetar los datos de forma manual si son normal o presentan alguna anomalía. Pero el análisis siguiente presenta un problema: Debido a la gran cantidad de datos y muchos de estos presentan patrones semejantes, etiquetar todos estos resultaría complicado y requeriría mucho tiempo

In [ ]:
import pandas as pd
from IPython.display import clear_output

def manual_labeling(unlabeled_data:pd.DataFrame, columns_selected:List[str], _max:int=20):
  for idx in unlabeled_data.index[:_max]:
    clear_output(wait=True) 
    data = unlabeled_data.loc[idx] 
    
    # Mostrar información del log no etiquetado
    max_len_columns_name = max([len(col_name) for col_name in columns_selected])
    for col_name in columns_selected:
      print(f"{col_name:>{max_len_columns_name}}: {data[col_name]}")
    
    option = input("\n0=Normal, 1=Anomalía, s=Saltar, q=Salir: ").strip()

    match option:
      case '0':
        df.at[idx, 'anomaly'] = 0
      case '1':
        df.at[idx, 'anomaly'] = 1 
      case 'q':
        break 

  return unlabeled_data

df_manual = manual_labeling(
  unlabeled_data, 
  columns_selected=[
    'ip_client', 
    'ident',
    'auth_user',
    'status', 
    'size', 
    'user_agent', 
    'method', 
    'url', 
    'protocol',
  ], 
  _max=100
)

In [ ]:
display(df_manual.head(10))
idx_update = df_manual[df_manual['anomaly'] != -1].index
df.loc[idx_update, 'anomaly'] = df_manual.loc[idx_update, 'anomaly']

### Etiquetado Manual por Lotes y Patrones Comunes

In [ ]:
display(df['anomaly'].value_counts())
labeled_data = df[df['anomaly'] != -1]
unlabeled_data = df[df['anomaly'] == -1]

In [ ]:
unlabeled_data = unlabeled_data.assign(
  n_tuple=list(
    zip(
      unlabeled_data['status'],
      unlabeled_data['method'],
      unlabeled_data['url'],
      unlabeled_data['protocol'],
    )
  )
)

In [ ]:
def etiquetar_por_tuplas(unlabeled_df):
  "Etiqueta n-tuplas formadas por status, method, url, protocol"
  # Contar frecuencia de n-tuplas
  tuple_counts = unlabeled_df['n_tuple'].value_counts()
  labels_map = {}

  for i, (tup, freq) in enumerate(tuple_counts.items(), 1):
    clear_output(wait=True) 
    status, method, url, protocol = tup

    # Mostrar información
    print(f"  N-tupla {i}/{len(tuple_counts)}")
    print(f" Frecuencia: {freq} logs")
    print(f" Características")
    print(f"  - status:   {status}")
    print(f"  - method:   {method}")
    print(f"  - URL:      {url}")
    print(f"  - protocol: {protocol}")
    
    option = input("\n   0=Normal, 1=Anomalía, s=Saltar, q=Salir: ").strip()

    match option:
      case '0':
        labels_map[tup] = 0
      case '1':
        labels_map[tup] = 1
      case 'q':
        break 
  
  return labels_map

tuple_labels = etiquetar_por_tuplas(unlabeled_data)

In [ ]:
if tuple_labels:
  # Crear columna temporal con mapeo
  unlabeled_data['temp_label'] = unlabeled_data['n_tuple'].map(tuple_labels)
  # Aplicar donde haya etiqueta
  mask = unlabeled_data['temp_label'].notna()
  unlabeled_data.loc[mask, 'anomaly'] = unlabeled_data.loc[mask, 'temp_label']
  # Estadísticas
  labeled_count = mask.sum()
  print(f"> {labeled_count} registros etiquetados")
  # Limpiar columnas temporales
  unlabeled_data = unlabeled_data.drop(['n_tuple', 'temp_label'], axis=1)

df_manual = pd.concat([labeled_data, unlabeled_data]).sort_index()
idx_update = df_manual[df_manual['anomaly'] != -1].index
df.loc[idx_update, 'anomaly'] = df_manual.loc[idx_update, 'anomaly']
display(df['anomaly'].value_counts())

### Guardar Dataset

In [ ]:
df.to_csv(log_prep_path, index=False)